In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
# df_train['genre_id'].value_counts()

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
    [torchvision.transforms.CenterCrop((64,960))],)



In [23]:
import torchaudio

### Make noisy label dataset

In [24]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'ecaresnext50t_32x4d_CQT_512_64_MIXUP'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


C:\Users\User\AppData\Local\Temp\ipykernel_13848\699689122.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13848\699689122.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13848\699689122.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13848\699689122.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users

In [25]:
f = np.zeros((len(df_test_valid),19))
for fld in range(5):
    for kernels in [
        'ecaresnext50t_32x4d_CQT_512_64_MIXUP'
                   ]:
        fname = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)
        fname = np.concatenate(fname)
         
        f += fname/5
        
f = pd.DataFrame(f)
f['ID'] = df_test_valid['song_id']
f.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.202314,0.356179,0.032742,0.094230,0.005089,0.056123,0.007655,0.102506,0.067761,0.002305,0.063208,0.004802,0.000163,0.002299,0.000662,0.000134,0.000316,0.001497,1.448700e-05,7072
1,0.003173,0.000726,0.000066,0.001137,0.000496,0.001014,0.000193,0.000902,0.000499,0.000592,0.000180,0.001804,0.987854,0.001259,0.000021,0.000006,0.000006,0.000064,6.602305e-06,10207
2,0.102968,0.030763,0.003730,0.668163,0.001219,0.002185,0.045660,0.089837,0.029530,0.000134,0.014461,0.000022,0.006090,0.004822,0.000130,0.000002,0.000264,0.000020,8.891908e-08,20008
3,0.638641,0.012932,0.001828,0.025384,0.013506,0.003503,0.210694,0.004850,0.013572,0.008785,0.055466,0.000684,0.007211,0.001765,0.000098,0.000756,0.000034,0.000148,1.422273e-04,10924
4,0.210956,0.061333,0.054925,0.042976,0.516081,0.006158,0.049330,0.008101,0.010671,0.014278,0.008153,0.001573,0.000129,0.001937,0.000575,0.007101,0.004438,0.001245,4.051081e-05,21896


In [26]:
consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)
consolNoisy.head()

C:\Users\User\AppData\Local\Temp\ipykernel_13848\4161917736.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.021872,0.309730,0.239648,0.055011,0.018603,0.053981,0.004016,0.091849,0.072898,0.016476,0.015736,0.022793,0.002309,0.040134,0.005212,0.003983,0.007569,1.816122e-02,0.000019,10150
1,0.018905,0.003669,0.001466,0.011473,0.002580,0.011708,0.001155,0.047774,0.004056,0.681420,0.000835,0.026047,0.002983,0.007012,0.166996,0.009163,0.000243,1.547105e-03,0.000969,11198
2,0.090427,0.227219,0.099546,0.008230,0.013731,0.045099,0.003970,0.013096,0.460174,0.003015,0.008401,0.000542,0.000203,0.000829,0.009013,0.001075,0.000022,1.528011e-02,0.000127,13166
3,0.011142,0.000195,0.000211,0.000941,0.976137,0.000305,0.001552,0.000111,0.001903,0.001832,0.000567,0.000092,0.004835,0.000012,0.000004,0.000006,0.000027,1.703607e-07,0.000130,12275
4,0.021869,0.034649,0.003807,0.759091,0.000768,0.017158,0.003321,0.100224,0.004456,0.003016,0.015527,0.014471,0.000418,0.012332,0.000058,0.000001,0.008029,7.372465e-04,0.000069,24102


In [27]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df,
                 noisyDF = consolNoisy,
                 train = True,
                 valid = False,
                 transform=tfms,

                ):
        
        self.df = df
        self.train = train
        self.valid = valid
        self.transform = transform
        self.noisyDF = noisyDF
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/train/{filename}')
        elif self.valid:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/train/{filename}')
        else:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/test/{filename}')
            
        return CQT


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            labels2 = self.noisyDF[self.noisyDF.ID==row.song_id].drop('ID',1).values[0]
            label = torch.tensor(row.genre_id,dtype=torch.long)
            label2 = torch.tensor(labels2).float()
            return samples, (label,label2)
        if self.valid:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:
            return samples

#### Loss

In [28]:
class CombinedLoss:
    "Dice and Focal combined"
    def __init__(self, axis=1, smooth=1., alpha=0.5):
        store_attr()
        self.focal_loss = CrossEntropyLossFlat()
        self.dice_loss =  BCEWithLogitsLossFlat()
        
    def __call__(self, pred, targ):
        if type(targ) is list:
            loss = self.focal_loss(pred, targ[0])*(1-self.alpha) + self.alpha * self.dice_loss(pred, targ[1])
        else:
            loss = self.focal_loss(pred, targ)
        return loss
    
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

#### FastAI model

In [29]:
import timm
# timm.create_model('resnext50d_32x4d', pretrained=False, in_chans=1)

In [30]:
NCLASS = df_train['genre_id'].nunique()
print(NCLASS)

19


In [31]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'ecaresnext50t_32x4d_CQT_512_64_Noisy'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit =  CombinedLoss()#LabelSmoothingCrossEntropy(0.02) #CrossEntropyLossFlat() #FocalLoss() #CrossEntropyLossFlat() #FocalLoss() #LabelSmoothingCrossEntropy(0.01)
        batch_size = 64
        n_epochs = 15

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,train=False,valid=True)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('ecaresnext50t_32x4d', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,NCLASS)
    
        f1_score = F1Score(average="micro")

        learn = Learner(dls, model,opt_func=Adam,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True,),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
        learn.fit_one_cycle(n_epochs, 1e-2,wd=1e-3)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False,
                           valid=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.013194,2.580044,0.254897,08:48
1,0.991449,3.155910,0.244601,05:01
2,0.936742,2.845794,0.285786,04:57
3,0.885594,2.510927,0.310899,04:57
4,0.836805,2.853840,0.335510,04:57
5,0.793747,1.956266,0.418634,04:57
6,0.752671,2.025236,0.397288,04:58
7,0.711795,1.765656,0.467604,05:01
8,0.667234,1.639757,0.496986,05:04
9,0.618408,1.604889,0.509041,05:10


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.014598,3.885952,0.124309,04:58
1,0.976581,2.336665,0.309643,04:59
2,0.930466,1.970592,0.331492,05:03
3,0.876966,2.031500,0.321195,05:04
4,0.833699,1.833983,0.393270,05:20
5,0.792802,1.615319,0.479407,06:27
6,0.752314,1.774543,0.455550,05:43
7,0.713672,1.685505,0.479910,06:28
8,0.668642,1.649374,0.487946,06:23
9,0.623129,1.706475,0.469864,06:28


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.018922,3.482354,0.127323,05:07
1,0.988020,2.467251,0.239327,04:59
2,0.939238,2.845168,0.208187,04:58
3,0.888788,1.945662,0.377449,04:59
4,0.841548,1.755825,0.427675,04:58
5,0.800088,1.630242,0.470367,04:58
6,0.763205,1.680935,0.449021,04:58
7,0.726137,1.610106,0.469111,04:58
8,0.684006,1.594189,0.481668,04:58
9,0.635004,1.538544,0.500502,04:58


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.026682,2.516506,0.237880,05:08
1,0.986927,3.251928,0.266516,04:57
2,0.932627,2.641129,0.323034,04:57
3,0.877242,2.270892,0.345391,04:57
4,0.833675,1.911506,0.393620,04:57
5,0.797134,1.637443,0.445114,04:57
6,0.758016,1.602748,0.467973,04:58
7,0.719977,1.565172,0.487566,04:58
8,0.679242,1.662420,0.477518,04:57
9,0.629855,1.712822,0.476262,04:57


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.010103,2.993249,0.216277,05:34
1,0.978577,2.224835,0.277568,06:44
2,0.921663,1.744427,0.424014,04:58
3,0.870729,1.860447,0.401155,04:57
4,0.825207,2.225466,0.354434,04:58
5,0.787390,2.120667,0.359457,04:57
6,0.749228,1.836123,0.420246,04:57
7,0.709426,1.541016,0.490329,04:57
8,0.666842,1.679071,0.467219,04:58
9,0.620141,1.496671,0.519719,04:57


In [34]:
df_test_valid.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


In [35]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

### OOF

In [36]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'ecaresnext50t_32x4d_CQT_512_64_Noisy'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [37]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [38]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [39]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.5651999196302994

In [40]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,1964,166,55,201,284,20,376,113,141,95,376,2,4,27,1,35,1,2,5
1,139,1680,761,158,29,156,12,128,228,38,32,8,2,29,24,10,5,9,0
2,35,674,1596,43,14,16,5,24,60,18,3,0,0,6,9,3,1,2,0
3,160,134,54,982,30,79,70,143,83,33,71,24,7,37,2,0,30,4,2
4,173,22,15,39,1312,9,23,7,25,21,19,0,0,1,1,6,2,4,1
5,19,105,20,60,4,766,0,43,83,30,6,13,0,8,26,3,9,8,0
6,194,13,2,29,20,2,645,6,15,1,29,0,0,4,1,4,0,0,0
7,106,81,11,94,3,44,10,450,25,8,91,44,0,31,2,1,1,3,0
8,82,120,36,44,27,49,10,16,215,18,7,1,2,5,9,9,1,4,3


### Mode based sub

In [41]:
kernel_type = 'ecaresnext50t_32x4d_CQT_512_64_Noisy'

In [42]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [43]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub: check OOF Score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop512_Mels128',
#                 'seresnet50t_MonoSpec_Hop448_Mels160',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

### Mode based sub

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_8bestmodels_CV_5862.csv',index=False)

In [ ]:
kernel_type

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
#                 'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_5bestmodels_CV_5877.csv',index=False)

### Avg probs

In [ ]:
# np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_0_probs.npy',allow_pickle=True)).shape,
df_test_valid.head()



In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_probEns_5bestmodels_NoCVInfo.csv',index=False)

#### Blind sub

In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/BlindSub_NoCVInfo.csv',index=False)

### End ###